##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recomendación de películas: modelos de recomendación en TFX

Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/recommenders"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/recommenders.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/recommenders.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/recommenders.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

## Tutorial de TFRS trasladado a TFX

Esta es una adaptación de un tutorial básico de TensorFlow Recommenders (TFRS) para TFX, que se diseñó para demostrar cómo usar TFRS en una canalización de TFX. Refleja el [tutorial básico](https://www.tensorflow.org/recommenders/examples/basic_retrieval).

Por contexto, los sistemas de recomendación del mundo real a menudo se componen de dos etapas:

1. La etapa de recuperación es responsable de seleccionar un conjunto inicial de cientos de candidatos entre todos los candidatos posibles. El objetivo principal de este modelo es eliminar de manera eficiente todos los candidatos que no le interesan al usuario. Debido a que el modelo de recuperación puede tratar con millones de candidatos, tiene que ser computacionalmente eficiente.
2. La etapa de clasificación toma los resultados del modelo de recuperación y los afina para seleccionar el mejor conjunto posible de recomendaciones. Su tarea consiste en reducir el conjunto de artículos que pueden interesar al usuario a una lista de posibles candidatos.

En este tutorial, nos centraremos en la primera etapa, la recuperación. Los modelos de recuperación suelen estar compuestos por dos submodelos:

1. Un modelo de consulta que calcula la representación de la consulta (normalmente un vector de incorporación de dimensionalidad fija) a partir de las características de consulta.
2. Un modelo candidato que calcula la representación candidata (un vector del mismo tamaño) mediante las características candidatas.

Luego, las salidas de los dos modelos se multiplican para obtener una puntuación de afinidad entre la consulta y el candidato, donde las puntuaciones más altas expresan una mejor coincidencia entre el candidato y la consulta.

En este tutorial, compilaremos y entrenaremos un modelo de dos torres utilizando el conjunto de datos Movielens.

Haremos lo siguiente:

1. Ingeriremos e inspeccionaremos el conjunto de datos MovieLens.
2. Implementaremos un modelo de recuperación.
3. Entrenaremos y exportaremos el modelo.
4. Haremos predicciones

## El conjunto de datos

El conjunto de datos Movielens es un conjunto de datos clásico del grupo de investigación [GroupLens](https://grouplens.org/datasets/movielens/) de la Universidad de Minnesota. Contiene un conjunto de calificaciones que un conjunto de usuarios otorgó a películas y es un caballo de batalla en la investigación de sistemas de recomendación.

Los datos se pueden tratar de dos formas:

1. Puede interpretarse como que expresa qué películas vieron (y calificaron) los usuarios y cuáles no. Esta es una forma de retroalimentación implícita, donde las visualizaciones de los usuarios nos dicen qué cosas prefieren ver y cuáles preferirían no ver.
2. También se puede considerar que expresa cuánto les gustaron a los usuarios las películas que vieron. Esta es una forma de retroalimentación explícita; como el usuario miró una película, podemos saber aproximadamente cuánto le gustó al observar la calificación que le dio.

En este tutorial, nos centramos en un sistema de recuperación: un modelo que predice un conjunto de películas del catálogo que es probable que vea el usuario. A menudo, los datos implícitos son más útiles aquí, por lo que trataremos a Movielens como un sistema implícito. Esto significa que cada película que vio un usuario es un ejemplo positivo y cada película que no vio es un ejemplo negativo implícito.

## Importaciones

Primero, eliminemos nuestras importaciones.

In [ ]:
!pip install -Uq tfx
!pip install -Uq tensorflow-recommenders
!pip install -Uq tensorflow-datasets

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe reiniciar el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...). Esto se debe a la forma en que Colab carga los paquetes.

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import apache_beam as beam

from absl import logging

from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Cómo crear un ExampleGen de TFDS

Creamos un [componente ExampleGen personalizado](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen) que usamos para cargar un conjunto de datos TensorFlow Datasets (TFDS). Esto usa un ejecutor personalizado en FileBasedExampleGen.

In [ ]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.Pipeline)
@beam.typehints.with_output_types(tf.train.Example)
def _TFDatasetToExample(  # pylint: disable=invalid-name
    pipeline: beam.Pipeline,
    exec_properties: Dict[str, Any],
    split_pattern: str
    ) -> beam.pvalue.PCollection:
    """Read a TensorFlow Dataset and create tf.Examples"""
    custom_config = json.loads(exec_properties['custom_config'])
    dataset_name = custom_config['dataset']
    split_name = custom_config['split']

    builder = tfds.builder(dataset_name)
    builder.download_and_prepare()

    return (pipeline
            | 'MakeExamples' >> tfds.beam.ReadFromTFDS(builder, split=split_name)
            | 'AsNumpy' >> beam.Map(tfds.as_numpy)
            | 'ToDict' >> beam.Map(dict)
            | 'ToTFExample' >> beam.Map(utils.dict_to_example)
            )

class TFDSExecutor(BaseExampleGenExecutor):
  def GetInputSourceToExamplePTransform(self) -> beam.PTransform:
    """Returns PTransform for TF Dataset to TF examples."""
    return _TFDatasetToExample

## Contexto de canalización de inicio de TFX

In [ ]:
context = InteractiveContext()

## Cómo preparar el conjunto de datos

Usaremos nuestro ejecutor personalizado en `FileBasedExampleGen` para cargar nuestros conjuntos de datos desde TFDS. Como tenemos dos conjuntos de datos, crearemos dos componentes `ExampleGen`.

In [ ]:
# Ratings data.
ratings_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-ratings', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(ratings_example_gen, enable_cache=True)

In [ ]:
# Features of all the available movies.
movies_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-movies', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(movies_example_gen, enable_cache=True)

## Cómo crear la utilidad `inspect_examples`

Creamos una utilidad conveniente para inspeccionar conjuntos de datos de TF.Examples. El conjunto de datos de calificaciones devuelve un diccionario de identificación de películas, identificación de usuarios, calificación asignada, marca de tiempo, información de la película e información del usuario:

In [ ]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

inspect_examples(ratings_example_gen)

El conjunto de datos de películas contiene la identificación de la película, el título de la película y datos sobre los géneros a los que pertenece. Tenga en cuenta que los géneros están codificados con etiquetas de números enteros.

In [ ]:
inspect_examples(movies_example_gen)

## ExampleGen generó la división

Cuando ingerimos el conjunto de datos de la lente de la película, nuestro componente `ExampleGen` dividió los datos en divisiones `train` y `eval`. En realidad, se denominan `Split-train` y `Split-eval`. Por defecto, la división es 66 % para entrenamiento y 34 % para evaluación.

## Cómo generar estadísticas de películas y calificaciones

Para una canalización de TFX, necesitamos generar estadísticas para el conjunto de datos. Lo hacemos usando un [componente StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen). Estos serán utilizados por el [componente SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) a continuación cuando generemos un esquema para nuestro conjunto de datos. De todos modos, esta es una buena práctica, porque es importante examinar y analizar los datos de forma continua. Como tenemos dos conjuntos de datos, crearemos dos componentes StatisticsGen.

In [ ]:
movies_stats_gen = tfx.components.StatisticsGen(
    examples=movies_example_gen.outputs['examples'])
context.run(movies_stats_gen, enable_cache=True)

In [ ]:
context.show(movies_stats_gen.outputs['statistics'])

In [ ]:
ratings_stats_gen = tfx.components.StatisticsGen(
    examples=ratings_example_gen.outputs['examples'])
context.run(ratings_stats_gen, enable_cache=True)

In [ ]:
context.show(ratings_stats_gen.outputs['statistics'])

## Cómo crear esquemas para películas y calificaciones

Para una canalización de TFX, necesitamos generar un esquema de datos a partir de nuestro conjunto de datos. Lo hacemos utilizando un [componente SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen). Esto será utilizado por el [componente Transform](https://www.tensorflow.org/tfx/guide/transform) a continuación para realizar nuestra ingeniería de características de una manera que sea altamente escalable a grandes conjuntos de datos y evite el sesgo entrenamiento/servicio. Como tenemos dos conjuntos de datos, crearemos dos componentes SchemaGen.

In [ ]:
movies_schema_gen = tfx.components.SchemaGen(
    statistics=movies_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(movies_schema_gen, enable_cache=True)

In [ ]:
context.show(movies_schema_gen.outputs['schema'])

In [ ]:
ratings_schema_gen = tfx.components.SchemaGen(
    statistics=ratings_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(ratings_schema_gen, enable_cache=True)

In [ ]:
context.show(ratings_schema_gen.outputs['schema'])

## Ingeniería de características usando Transform

Para un diseño estructurado y repetible de una canalización de TFX, necesitaremos un enfoque escalable para la ingeniería de características. Esto nos permite manejar grandes conjuntos de datos que suelen formar parte de muchos sistemas de recomendación y también evita el sesgo entre entrenamiento y servicio. Lo haremos con ayuda del [componente Transform](https://www.tensorflow.org/tfx/guide/transform).

El componente Transform usa un archivo de módulo para proporcionar código de usuario para la ingeniería de características que queremos hacer, por lo que nuestro primer paso es crear ese archivo de módulo. Como tenemos dos conjuntos de datos, crearemos dos de estos archivos de módulo y dos componentes Transform.

Una de las cosas que nuestro recomendador necesita son vocabularios para los campos `user_id` y `movie_title`. En el [tutorial de basic_retrieval,](https://www.tensorflow.org/recommenders/examples/basic_retrieval) estos se crean con Numpy en línea, pero aquí usaremos Transform.

Nota: La magia de celda `%%writefile {_movies_transform_module_file}` a continuación crea y escribe el contenido de esa celda en un archivo en el servidor de bloc de notas donde se ejecuta este bloc de notas (por ejemplo, Colab VM). Al hacer esto fuera de un bloc de notas, simplemente crearía un archivo Python.

In [ ]:
_movies_transform_module_file = 'movies_transform_module.py'

In [ ]:
%%writefile {_movies_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  # We only want the movie title
  return {'movie_title':inputs['movie_title']}

In [ ]:
movies_transform = tfx.components.Transform(
    examples=movies_example_gen.outputs['examples'],
    schema=movies_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_movies_transform_module_file))
context.run(movies_transform, enable_cache=True)

In [ ]:
context.show(movies_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(movies_transform, channel_name='transformed_examples')

In [ ]:
_ratings_transform_module_file = 'ratings_transform_module.py'

In [ ]:
%%writefile {_ratings_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import pdb

NUM_OOV_BUCKETS = 1

def preprocessing_fn(inputs):
  # We only want the user ID and the movie title, but we also need vocabularies
  # for both of them.  The vocabularies aren't features, they're only used by
  # the lookup.
  outputs = {}
  outputs['user_id'] = tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1')
  outputs['movie_title'] = tft.sparse_tensor_to_dense_with_shape(inputs['movie_title'], [None, 1], '-1')

  tft.compute_and_apply_vocabulary(
      inputs['user_id'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='user_id_vocab')

  tft.compute_and_apply_vocabulary(
      inputs['movie_title'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='movie_title_vocab')

  return outputs

In [ ]:
ratings_transform = tfx.components.Transform(
    examples=ratings_example_gen.outputs['examples'],
    schema=ratings_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_ratings_transform_module_file))
context.run(ratings_transform, enable_cache=True)

In [ ]:
context.show(ratings_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(ratings_transform, channel_name='transformed_examples')

## Implementación de un modelo en TFX

En el tutorial de [basic_retrieval](https://www.tensorflow.org/recommenders/examples/basic_retrieval), el modelo se creó en línea en el tiempo de ejecución de Python. En una canalización de TFX, el modelo, la métrica y la pérdida se definen y entrenan en el archivo del módulo para un [componente de canalización conocido como Trainer](https://www.tensorflow.org/tfx/guide/trainer). Esto hace que el modelo, la métrica y la pérdida formen parte de un proceso repetible que se puede automatizar y monitorear.

### Arquitectura del modelo de TensorFlow Recommenders

Vamos a compilar un modelo de recuperación de dos torres. El concepto de dos torres significa que tendremos una torre de consultas que calculará la representación del usuario a partir de las características del usuario y otra torre de elementos que calculará la representación de la película a partir de las características de la película. Podemos construir cada torre por separado (en los métodos `_build_user_model()` y `_build_movie_model()` siguientes) y luego combinarlas en el modelo final (como en la clase `MobieLensModel`). `MovieLensModel` es una subclase de la clase base `tfrs.Model`, que agiliza la compilación de modelos: todo lo que necesitamos hacer es configurar los componentes en el método `__init__` e implementar el método `compute_loss`, tomando las características sin procesar y devolviendo un valor de pérdida.

In [ ]:
# We're now going to create the module file for Trainer, which will include the
# code above with some modifications for TFX.

_trainer_module_file = 'trainer_module.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import Dict, List, Text

import pdb

import os
import absl
import datetime
import glob
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_recommenders as tfrs

from absl import logging
from tfx.types import artifact_utils

from tfx import v1 as tfx
from tfx_bsl.coders import example_coder
from tfx_bsl.public import tfxio

absl.logging.set_verbosity(absl.logging.INFO)

EMBEDDING_DIMENSION = 32
INPUT_FN_BATCH_SIZE = 1


def extract_str_feature(dataset, feature_name):
  np_dataset = []
  for example in dataset:
    np_example = example_coder.ExampleToNumpyDict(example.numpy())
    np_dataset.append(np_example[feature_name][0].decode())
  return tf.data.Dataset.from_tensor_slices(np_dataset)


class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, tf_transform_output, movies_uri):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model

    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'movie_title')

    loss_metrics = tfrs.metrics.FactorizedTopK(
        candidates=movies_dataset.batch(128).map(movie_model)
        )

    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=loss_metrics
        )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    try:
      user_embeddings = tf.squeeze(self.user_model(features['user_id']), axis=1)
      # And pick out the movie features and pass them into the movie model,
      # getting embeddings back.
      positive_movie_embeddings = self.movie_model(features['movie_title'])

      # The task computes the loss and the metrics.
      _task = self.task(user_embeddings, positive_movie_embeddings)
    except BaseException as err:
      logging.error('######## ERROR IN compute_loss:\n{}\n###############'.format(err))

    return _task


# This function will apply the same transform operation to training data
# and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  try:
    transformed_features = tft_layer(raw_features)
  except BaseException as err:
    logging.error('######## ERROR IN _apply_preprocessing:\n{}\n###############'.format(err))

  return transformed_features


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  try:
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size),
      tf_transform_output.transformed_metadata.schema)
  except BaseException as err:
    logging.error('######## ERROR IN _input_fn:\n{}\n###############'.format(err))

  return None


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""
  try:
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
      """Returns the output to be used in the serving signature."""
      try:
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        result = model(transformed_features)
      except BaseException as err:
        logging.error('######## ERROR IN serve_tf_examples_fn:\n{}\n###############'.format(err))
      return result
  except BaseException as err:
      logging.error('######## ERROR IN _get_serve_tf_examples_fn:\n{}\n###############'.format(err))

  return serve_tf_examples_fn


def _build_user_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to ratings
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the query tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_user_ids = tf_transform_output.vocabulary_by_name('user_id_vocab')
    users_vocab_str = [b.decode() for b in unique_user_ids]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=users_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(users_vocab_str) + 1, embedding_dimension)
         ])
  except BaseException as err:
    logging.error('######## ERROR IN _build_user_model:\n{}\n###############'.format(err))

  return model


def _build_movie_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to movies
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the candidate tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_movie_titles = tf_transform_output.vocabulary_by_name('movie_title_vocab')
    titles_vocab_str = [b.decode() for b in unique_movie_titles]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=titles_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(titles_vocab_str) + 1, embedding_dimension)
        ])
  except BaseException as err:
      logging.error('######## ERROR IN _build_movie_model:\n{}\n###############'.format(err))
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  try:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, INPUT_FN_BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                            tf_transform_output, INPUT_FN_BATCH_SIZE)

    model = MovielensModel(
        _build_user_model(tf_transform_output, EMBEDDING_DIMENSION),
        _build_movie_model(tf_transform_output, EMBEDDING_DIMENSION),
        tf_transform_output,
        fn_args.custom_config['movies']
        )

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
  except BaseException as err:
    logging.error('######## ERROR IN run_fn before fit:\n{}\n###############'.format(err))

  try:
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['epochs'],
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during fit:\n{}\n###############'.format(err))

  try:
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    movies_artifact = fn_args.custom_config['movies'].get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'eval')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")

    movies_dataset = extract_str_feature(movies, 'movie_title')

    index.index_from_dataset(
      tf.data.Dataset.zip((
          movies_dataset.batch(100),
          movies_dataset.batch(100).map(model.movie_model))
      )
    )

    # Run once so that we can get the right signatures into SavedModel
    _, titles = index(tf.constant(["42"]))
    print(f"Recommendations for user 42: {titles[0, :3]}")

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(index,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    index.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

  except BaseException as err:
      logging.error('######## ERROR IN run_fn during export:\n{}\n###############'.format(err))

## Cómo entrenar el modelo

Después de definir el modelo, podemos ejecutar el [componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para ejecutar el entrenamiento del modelo.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    examples=ratings_transform.outputs['transformed_examples'],
    transform_graph=ratings_transform.outputs['transform_graph'],
    schema=ratings_transform.outputs['post_transform_schema'],
    train_args=tfx.proto.TrainArgs(num_steps=500),
    eval_args=tfx.proto.EvalArgs(num_steps=10),
    custom_config={
        'epochs':5,
        'movies':movies_transform.outputs['transformed_examples'],
        'movie_schema':movies_transform.outputs['post_transform_schema'],
        'ratings':ratings_transform.outputs['transformed_examples'],
        'ratings_schema':ratings_transform.outputs['post_transform_schema']
        })

context.run(trainer, enable_cache=False)

## Cómo exportar el modelo

Después de entrenar el modelo, podemos usar el [componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) para exportarlo.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/tfrs_retrieval')

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

## Hacer predicciones

Ahora que tenemos un modelo, lo volvemos a cargar y hacemos predicciones.

In [ ]:
loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")

## Siguientes pasos

En este tutorial, aprendimos a implementar un modelo de recuperación con TensorFlow Recommenders y TFX. Para ampliar lo que se presenta aquí, eche un vistazo al tutorial de [clasificación de TFRS con TFX](https://www.tensorflow.org/recommenders/examples/ranking_tfx).